In [1]:
import sys

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import tensorflow as tf
import optuna
import dataset
import numpy as np
import keras

2024-01-28 19:20:14.680016: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 19:20:14.704462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 19:20:14.704484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 19:20:14.705391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 19:20:14.710261: I tensorflow/core/platform/cpu_feature_guar

## create dataset


In [2]:
BATCH_SIZE = 64
train_images_rgx = "../../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"

train_dataset, val_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE
)

2024-01-28 19:20:15.606957: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 19:20:15.631500: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 19:20:15.631619: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Train Dataset: <_ParallelBatchDataset element_spec=(TensorSpec(shape=(64, 256, 256, 3), dtype=tf.float16, name=None), TensorSpec(shape=(64, 5), dtype=tf.float32, name=None))>
Val Dataset: <_ParallelBatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float16, name=None), TensorSpec(shape=(None, 5), dtype=tf.float32, name=None))>


L355
2024-01-28 19:20:15.632846: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 19:20:15.632892: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 19:20:16.304677: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#

## define create model function


In [3]:
from keras import layers

layer_options = ["Conv2D", "Depthwise2D", "Pointwise2D"]
activation_options = ["linear", "relu", "tanh"]
filter_size_options = [3, 4, 5, 8, 16]


def clip_filter_size(fs, ih):
    return min(fs, ih)


def create_layer(trial, layer_type, layer_id, max_stride, input_h_size):
    if layer_type == "Conv2D":
        num_filters = trial.suggest_int(f"layer_{layer_id}_conv2d_filters", 4, 32, step=4)
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_conv2d_fsize", filter_size_options
        )

        stride_size = trial.suggest_int(f"layer_{layer_id}_conv2d_ssize", 1, max_stride)
        layer = layers.Conv2D(
            num_filters, clip_filter_size(filter_size, input_h_size), stride_size
        )

    elif layer_type == "Depthwise2D":
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_dwconv2d_fsize", filter_size_options
        )
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_dwconv2d_ssize", 1, max_stride
        )
        layer = layers.DepthwiseConv2D(
            clip_filter_size(filter_size, input_h_size), stride_size
        )

    elif layer_type == "Pointwise2D":
        num_filters = trial.suggest_int(f"layer_{layer_id}_pwconv2d_filters", 4, 32, step=4)
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_pwconv2d_ssize", 1, max_stride
        )
        layer = layers.Conv2D(num_filters, 1, stride_size)

    return layer


def create_model(trial):
    # add the input layer
    x = x_in = layers.Input(shape=(256, 256, 3))

    # suggent a number of layer
    num_layers = trial.suggest_categorical("num_layers", [1, 2, 4])

    # limits the stride
    max_stride = 16 // num_layers

    # convolutional layers
    for i in range(num_layers):
        layer_selected = trial.suggest_categorical(f"layer_{i}", layer_options)
        layer = create_layer(
            trial, layer_selected, i, max_stride, input_h_size=x.shape[1]
        )
        x = layer(x)
        act_selected = trial.suggest_categorical(f"activation_{i}", activation_options)
        x = layers.Activation(act_selected)(x)

    # output layer
    x = layers.Flatten()(x)
    x = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(x_in, x)

# if testing


```
class MockTrial:
    def suggest_int(self, name, min, max, log=False):
        return np.random.randint(min, max)

    def suggest_categorical(self, name, options):
        print(name)
        return options[np.random.randint(0, len(options) - 1)]

    def suggest_discrete_uniform(self, name, min, max, q):
        return (np.random.randint(min, max) // q) * q

model = create_model(MockTrial())
model.build(input_shape=(BATCH_SIZE, 256, 256, 3))
model.summary()
```

In [4]:
def create_optimizer(trial):
    lr = trial.suggest_float("adam_learning_rate", 1e-5, 1e-1, log=True)
    optimizer = keras.optimizers.Adam(lr)
    return optimizer


def create_model_checkpointer(save_path):
    return keras.callbacks.ModelCheckpoint(
        save_path,
        "val_accuracy",
        mode="max",
        save_best_only=True,    
        save_weights_only=True    
    )


def crate_early_stopper():
    return keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0.1,
        patience=10,
        verbose=0,
        mode="max",                
        start_from_epoch=50,
    )


def objective(trial):
    EPOCHS = 100

    # Build model and optimizer.
    model = create_model(trial)
    assert isinstance(model, keras.Model), "model is not a model"

    optimizer = create_optimizer(trial)

    # compile the model
    model.compile(
        optimizer, loss=keras.losses.categorical_crossentropy, metrics=["accuracy"]
    )

    # crate model checkpointer
    saved_best_model_path = f"saved_models/best_model_t{trial._trial_id}/"
    model.save(saved_best_model_path, True, "tf") # save all model 

    model_chkp = create_model_checkpointer(saved_best_model_path)
    early_stopper = crate_early_stopper()

    # Training and validating cycle.
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=[model_chkp, early_stopper],
        verbose="0",
    )

    # load best model
    best_model = keras.models.load_model(saved_best_model_path)

    # Evaluate the model accuracy on the validation set.
    score = best_model.evaluate(val_dataset, verbose=0)

    # Return best model validation accuracy.
    return score[1]


def search():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [5]:
search()

[I 2024-01-28 19:20:16,541] A new study created in memory with name: no-name-9059ce5f-ba8c-4f2e-b0a1-719348a0b367
/tmp/ipykernel_926437/2502342074.py:14: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  num_filters = trial.suggest_int(f"layer_{layer_id}_conv2d_filters", 4, 32, 4)
/tmp/ipykernel_926437/2502342074.py:36: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  num_filters = trial.suggest_int(f"layer_{layer_id}_pwconv2d_filters", 4, 32, 4)


INFO:tensorflow:Assets written to: saved_models/best_model_t0/assets


INFO:tensorflow:Assets written to: saved_models/best_model_t0/assets


Epoch 1/100


2024-01-28 19:20:18.299371: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
